In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scipy
from matplotlib import rc
import matplotlib
import yfinance
import seaborn as sns
plt.style.use('seaborn')

In [ ]:
tickers = ["EEM", "IVV", "IEV", "IXN", "IYR", "IXG", "IYR", "GC=F", "HYG","TLT"]
#spx = pd.read_csv('../data/spx.csv').stack().tolist()

**Downloading data and calculating returns**

In [ ]:
raw_data = yfinance.download(tickers, start="2008-1-1", back_adjust=True, auto_adjust=True)
closing_prices = raw_data['Close']
returns = closing_prices.pct_change().iloc[1:].ffill()*100 #returns
index = closing_prices/closing_prices.iloc[0]*100 #index

**Plotter index figur med alle aktiver**

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=190)
ax.plot(index["EEM"], label="Emerging Markets", c="red",linewidth=1)
ax.plot(index["IVV"], label="S&P 500", c="grey",linewidth=1)
ax.plot(index["IEV"], label="MSCI Europe", c="green",linewidth=1)
ax.plot(index["IXN"], label="Global Tech", c="black",linewidth=1)
ax.plot(index["IYR"], label="US Real Estate", c="aqua",linewidth=1)
ax.plot(index["IXG"], label="Global Financials", c="orange",linewidth=1)
ax.plot(index["EXI"], label="Global Industrials", c="purple",linewidth=1)
ax.plot(index["GC=F"], label="Gold", c="Gold",linewidth=1)
ax.plot(index["BZ=F"], label="Brent Crude Oil", c="brown",linewidth=1)
ax.plot(index["HYG"], label="High-yield Bonds", c="blue",linewidth=1)
ax.plot(index["TLT"], label="20+ Year Treasuries", c="pink",linewidth=1)
ax.set_ylabel('Index (2008=index 100)',loc='top', rotation = 0, labelpad = -140) 
ax.legend()
fig.savefig("Stocks_index_plot.png", bbox_inches = 'tight', pad_inches = 0.1)

**Fitter fordelinger og laver PDF'er for udvalgte tickers**

In [ ]:
ticker="IXG"

mu, std = scipy.norm.fit(returns[ticker]) #normal distribution fi#t
beta, loc_ged, scale_ged = scipy.gennorm.fit(returns[ticker]) #GED f#it
df, nc_t, loc_t, scale_t = scipy.nct.fit(returns[ticker]) #skewed student's t distrubiton fit

#draw data
xmin = returns[ticker].min()
xmax = returns[ticker].max()
x = np.linspace(xmin, xmax, 1000)

normal_data=scipy.norm.pdf(x, mu, std)
GED_data=scipy.gennorm.pdf(x, beta,loc_ged,scale_ged)
t_data=scipy.nct.pdf(x, df,nc_t,loc_t,scale_t)

**Plotter PDF mod histogram af faktisk data**

In [ ]:
plt.style.use('seaborn')
fig, ax = plt.subplots(1, 1, figsize=(7, 7), dpi=190)
ax.plot(x, normal_data, label="Fitted gaussian")#, c="blue")
ax.plot(x, GED_data, label="Fitted GED")#, c = "black")
ax.plot(x, t_data, label ="Fitted skewed-t")#, c="red")
ax.hist(returns[ticker], bins=100, alpha=0.6, density = True)
#ax.set_xlim(-0.075, 0.075)
ax.legend()
ax.set_xlabel('Returns') 
ax.set_ylabel('Probability Density',loc='top', rotation = 0, labelpad = -110) 
#fig.savefig("Density.png", bbox_inches = 'tight', pad_inches = 0.1)

Fitter GARCH model og plotter data

In [ ]:
#indsæt GARCH del

In [ ]:
f, ax1 = plt.subplots(1, 1, figsize=(15,12))
ax1.plot(returns[ticker], alpha=.8)
ax1.set_ylabel('Yearly Returns',loc='top', rotation = 0, labelpad = -100)
ax1.set_ylim((-0.35,0.35))
#plt.savefig("Empirical timeseries.png", bbox_inches = 'tight', pad_inches = 0.1)

f, ax2 = plt.subplots(1, 1, figsize=(15,12))
ax2.plot(normal_data, alpha=.8)
ax2.set_ylabel('Yearly Returns',loc='top', rotation = 0, labelpad = -100)
ax2.set_ylim((-0.35,0.35))
#plt.savefig("Gaussian timeseries.png", bbox_inches = 'tight', pad_inches = 0.1)

f, ax3 = plt.subplots(1, 1, figsize=(15,12))
ax3.plot(GED_data, alpha=.8)
ax3.set_ylabel('Yearly Returns',loc='top', rotation = 0, labelpad = -100)
ax3.set_ylim((-0.35,0.35))
#plt.savefig("GED timeseries.png", bbox_inches = 'tight', pad_inches = 0.1)

f, ax4 = plt.subplots(1, 1, figsize=(15,12))
ax4.plot(t_data, alpha=.8)
ax4.set_ylabel('Yearly Returns',loc='top', rotation = 0, labelpad = -100)
ax4.set_ylim((-0.35,0.35))
#plt.savefig("Skewed Student's t timeseries.png", bbox_inches = 'tight', pad_inches = 0.1)

**Laver Jarque bera normalitets test**

In [86]:
descriptive_statistics

for ticker in tickers:
    teststatic, p_value= scipy.jarque_bera(returns[ticker])
    print(ticker,p_value)

EEM 0.0
IVV 0.0
IEV 0.0
GC=F 0.0
HYG 0.0
IXG 0.0
IYR 0.0
EXI 0.0
BZ=F 0.0
IXN 0.0
TLT 0.0
